<a href="https://colab.research.google.com/github/Huebr/OtimizacaoIrrestrita/blob/main/projeto2_on_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Programação Não Linear - Segundo Projeto Computacional
**Universidade Federal do Ceará**
Professor: RICARDO COELHO
Alunos: MARCIO BARROS OLIVEIRA DE SOUZA, 
        PAULINO CARDIAL RABELO e 
        PEDRO JORGE DE ABREU FIGUEREDO

#Introdução

Este trabalho consiste no desenvolvimento computacional de métodos de otimização não linear irrestrita. Dentre eles apresentamos o Método de Newton, Gradiente Descente, quase-newton Broyden, Fletcher, Goldfarb e Shanno(BFGS) e Método do Gradiente Conjugado Polak e Ribiére (PR). Primeiramente, apresentamos um pouco sobre o ambiente computacional usado. Em seguida, mostramos a implementação dos métodos em conjunto com um breve resumo de cada um deles. Nas seções "Modo de Usar" e "Parâmetros dos Métodos" mostramos como chamar os métodos e quais os parâmetros implementados. Por fim, fazemos um análise simples encima de um conjunto de dados disponibilizados para o desenvolvimento desta atividade.


#Instalação Ambiente

No desenvolvimento desta atividade íremos utilizar Julia como linguagem de implementação pelo seu facil uso e vasto conjunto de bibliotecas que oferecem suporte a otimização matemática. Para facilitar a distribuição fornecemos um notebook Jupyter que colocamos pode ser usado no Colab Google. Veja que facilmente podemos exportar o mesmo para o uso local.

Para podermos executar corretamente no ambiente Colab temos que executar a seguinte célula e reiniciar a aba do navegador aplicando "F5" quando a célula terminar de instalar o kenerl Julia. No caso do uso local basta ter a aplicação Jupyter Notebook (,no Windows Anaconda) e uma distribuição Julia 1.4.2 ou mais recente.

In [ ]:
# Installation cell
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


#Dependências

As depências a seguir devem ser instaladas para permitir importar as bibliotecas que são usadas neste trabalho.

In [ ]:
9#Julia 1.4 Environment
using Pkg
pkg"add ForwardDiff; precompile;"
pkg"add Plots; precompile;"
pkg"add PyPlot; precompile;"

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
Precompiling project...
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
Precompiling project...
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
Precompiling project...


<h1>Funções Auxiliares</h1>

Durante a implementação usamos uma biblioteca para calcular derivadas e gradientes através da técnica  de "Automatic Differentiation". Essa permite calcular com precisão e agilidade os métodos aqui utilizados. Além disso, usamos o framework Plots em conjunto com a engine PyPlot para gerar gráficos das funções testadas.



In [ ]:
import ForwardDiff  # calcula derivadas usando automatic differentiation in forward mode
using LinearAlgebra #adiciona operações de algebra linear
using Plots

pyplot()#usa PyPlot como backend engine
∇ = (h,x)->ForwardDiff.gradient(h,x) #gradiente de h(x)
Hessian = (h,x)->ForwardDiff.hessian(h,x) # hessian de h(x)
Df = (h,x)->ForwardDiff.derivative(h,x);  # primeira derivada de h'(x)
D_2f = (h,x)->ForwardDiff.derivative(z->ForwardDiff.derivative(h,z),x) # segunda derivada h"(x)

#72 (generic function with 1 method)

Em seguida mostramos as implementações dos métodos de busca linear que serão usados neste trabalho. Uma analise desses foi feita no [Projeto 1](https://colab.research.google.com/drive/1Nt6ffNGr-3ej0gCgIW2sG5DYSnJ7Pywh?usp=sharing).

> Bloco com recuo



In [ ]:
#função de busca linear
function linear_search(f::Function,x::Vector,d::Vector,method::Function)
    ϕ = t->f(x+t*d)
    α = method(f,x,d)
    return (ϕ(α),x+α*d)
end

function secao_aurea(f::Function,x::Vector,d::Vector;φ::Function = t->f(x+t*d),ρ = 1/2,ϵ = 1e-9)::Float64
    θ_1 = (3 - sqrt(5))/2.0
    θ_2 = (sqrt(5) - 1)/2.0
    #Obtenção intervalo
    a=0
    b=2*ρ
    s = b / 2
    while(φ(b)<φ(s))
        a = s
        s = b
        b *= 2
    end
    #Obtenção de t
    u = a + θ_1*(b - a)
    v = a + θ_2*(b - a)
    while(b - a > ϵ)
        if(φ(u)<φ(v))
            b = v
            v = u
            u = a + θ_1*(b - a)
        else
            a = u
            u = v
            v = a + θ_2*(b - a)
        end
    end
    return (u+v)/2
end

function secao_aurea(f::Function,x::Vector,d::Vector,(a,b)::Tuple{Float64,Float64};φ::Function = t->f(x+t*d),ϵ = 1e-9)::Float64
    θ_1 = (3 - sqrt(5))/2.0
    θ_2 = (sqrt(5) - 1)/2.0
    #Obtenção de t
    u = a + θ_1*(b - a)
    v = a + θ_2*(b - a)
    while(b - a > ϵ)
        if(φ(u)< φ(v))
            b = v
            v = u
            u = a + θ_1*(b - a)
        else
            a = u
            u = v
            v = a + θ_2*(b - a)
        end
    end
    return (u+v)/2
end

function newton(f::Function,x::Vector,d::Vector;φ::Function = y->f(x+y*d),t::Float64=1.0,ϵ = 1e-9,n_iter=1000)::Float64
   iter = 0
   while (abs(Df(φ,t)) > ϵ && iter < n_iter)
         t = t - (Df(φ,t)/D_2f(φ,t))
         iter = iter + 1
   end 
   return t
end

function armijo(f::Function,x::Vector,d::Vector;φ::Function = y->f(x+y*d),t::Float64= 1.0,η::Float64 = 0.25)
    # f(x + td) > f(x) +t(∇f(x)⋅ d)
    while (φ(t)>φ(0)+ η*t*(∇(f,x)⋅d))
      t *=0.8
    end
    return t
end

armijo (generic function with 1 method)

#Métodos de Otimização Irrestrita

##Metodo da Gradiente


O método gradiente é um método de descida de primeira ordem que gera uma sequência {$x_{k}$}, globalmente convergente, a partir de um $x_{0}$ qualquer, que irá convergir para $x^*$ .

\

\begin{equation}
x_{k+1}=x_{k}+t_{k}d_{k}=x_{k}-t_{k}\nabla f(x_{k})
\end{equation}
$t_{k}>0$








Na próxima imagem podemos ver um exemplo, onde começando do ponto $A$, e os vetores são os anti-gradiente do ponto, que vamos tomar como a direção para obter o próximo ponto.

<div>


Podemos ver na próxima imagem o caminho que a sequência de pontos leva até chegar ao mínimo, com $x_{0}=A$

<div>


Lema

  Se $t_{k}$ é obtido por uma minimização local de $f(x_{k}+t_{k}d_{k})$, então $(d_{k+1})^{T}d_{k}$=0

In [ ]:
#verificar se a tupla está funcionando corretamente
function gradient_descent(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)
    while(norm(∇(f,x))> ϵ)
      d = -∇(f,x)
      x = linear_search(f,x,d,linear_method)[2]
    end
    return (f(x),x)
end


gradient_descent (generic function with 1 method)

In [ ]:
gradient_descent(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[0,1],armijo)

(0.0, [2.0, 1.0])

# Metodo de Newton



O método de Newton serve para obter um $x^*\in\mathbb{R}^{n}$, tal que

\begin{equation}
\phi(x^*)=0
\end{equation}

onde $\phi:\mathbb{R}^n\rightarrow\mathbb{R}^n$ diferenciável. Portanto o método vai gerar uma sequência de pontos a partir de um $x_{0}$ que irá convergir para $x^*$.

Para construir a sequência vamos usar a série de Taylor.

\

\begin{equation}
\phi(x)=\phi(x_{k})+\nabla\phi(x_{k})(x-x_{k})=0
\end{equation}

\

Isolando $x$, obtemos

\

\begin{equation}
  x_{k+1}=x_{k}-(\nabla\phi(x_{k}))^{-1}\phi(x_{k})
\end{equation}

\

Agora iremos contextualizar o método de Newton para o nosso problema. Queremos achar um $x^*\in\mathbb{R}^{n}$, na qual $\nabla f(x^*)=0$. Assim, podemos considerar $\phi(x)=\nabla f(x)$, pois $\nabla f(x):\mathbb{R}^n\rightarrow\mathbb{R}^n$, logo ficaremos com a nossa sequência

\

\begin{equation}
  x_{k+1}=x_{k}-(\nabla^2f(x_{k}))^{-1}\nabla f(x_{k})
\end{equation}

\

Podemos considerar como método de descida com $t_{k}=1$ constante e $d_{k}=-(\nabla^2f(x_{k}))^{-1}\nabla f(x_{k})$.


Podemos ver na próxima imagem como o método funciona, com $\phi:\mathbb{R}\rightarrow\mathbb{R}$.

<div>


O próximo teorema fala que vai existir uma vizinhança do minimizador local $x^*$ de $f$ com $\nabla^2f(x^*)>0$, e se nosso $x_{0}$ pertencer a essa vizinhança, então o algoritmo do método de Newton com $t_{k}=1$ constante, gera uma sequência $\{x_{k}\}$ que converge superlinearmente para $x^*$, e $\nabla^2f(x_{k})>0$, $\forall k\in\mathbb{N}$, com convergência quadrática se $\nabla^2f$ for Lipchitz.

\

Teorema


Seja $f:\mathbb{R}^n\rightarrow\mathbb{R}$ de classe $c^2$. Suponha que $x^*\in\mathbb{R}^n$ seja um minimizador local de $f$, com $\nabla^2f(x^*)>0$. Então existe $\delta>0$ tal que se $x^{0}\in B(x^{*},\delta$), o algoritmo do método de Newton, com $t_{k}=1$ constante, gera uma sequência $(x_{k})$ tal que.

1. $\nabla^2f(x_{k})>0$, $\forall k\in\mathbb{N}$;
2. $(x_{k})$ converge superlinearmente para $x^*$;
3. Se $\nabla^2f$ é Lipschitz, então a convergência é quadrática;

Porem podemos tambem variar o passo $t_{k}$ e com isso temos o próximo teorema.

\

Teorema 

Suponha $\nabla^2 f(x)>0$, $\forall x\in \mathbb{R}^{n}$. Então o algoritmo do Método de Newton, com $t_{k}$ calculado pela busca exata ou pela busca de Armijo, é globalmente convergente.

In [ ]:
function newton_search(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)
    while(norm(∇(f,x))> ϵ)
        d = -inv(Hessian(f,x))*∇(f,x)
        x = linear_search(f,x,d,linear_method)[2]
    end
    return (f(x),x)
end

newton_search (generic function with 1 method)

In [ ]:
newton_search(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[0,1],armijo)

(0.0, [2.0, 1.0])

#Metodo de quase-newton BFGS

O método de quase-Newton BFGS tem como principal ponto determinar a inversa da matriz Hessiana de maneira aproximada. Portanto, o que teriamos como direção $d_{k}=-(\nabla^2f(x_{k}))^{-1}\nabla f(x_{k})$ no método de Newton, vamos ter $d_{k}=-H_{k}\nabla f(x_{k})$ no método de quase-Newton, onde $H_{k}\in\mathbb{R}^{n\times n}$ é a matriz inversa da Hessiana aproximada, que é definida positiva e simétrica para todo $k$.

Podemos fazer as aproximações colocando uma condição.

\

\begin{equation}
H_{k+1}q_{k}=p_{k}
\end{equation}

\

Onde $q_{k}=\nabla f(x_{k+1})-\nabla f(x_{k})$ e $p_{k}=x_{k+1}-x_{k}$.

\

O método BFGS obtem a matriz aproximada $H_{k+1}$, onde para todo $k$, $H_{k+1}$ vai ser definida como

\

\begin{equation}
  H_{k+1}=H_{k}+(1+\frac{(q_{k})^{\top}H_{k}q_{k}}{(p_{k})^{\top}q_{k}})\frac{p_{k}(p_{k})^{\top}}{(p_{k})^{\top}q_{k}}-\frac{p_{k}(q_{k})^{\top}H_{k}+H_{k}q_{k}(p_{k})^{\top}}{(p_{k})^{\top}q_{k}}
\end{equation}

sendo $H_{k}=B_{k}^{-1}$

In [ ]:
function quasi_newton_BFGS(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)
    H = I
     flag = true
    g_new=∇(f,x)
    while(norm(g_new)> ϵ)
       d = -H*g_new
       x_old=x
       g_old = g_new
      x=linear_search(f,x,d,linear_method)[2]
      p = x - x_old
      g_new = ∇(f,x)
      q = g_new - g_old
      qT = transpose(q)
      pT = transpose(p)
       if(flag)
          H = ((qT*p)/(qT*q))*I #scaling of I
          flag =false
      end
      H = H+ (1+ ((qT*H*q)/(pT*q)))*((p*pT)/(pT*q)) - ((p*qT*H + H*q*pT)/(pT*q))
    end
    return (f(x),x)
end

quasi_newton_BFGS (generic function with 1 method)

In [ ]:
quasi_newton_BFGS(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[0,1],armijo)

(0.0, [2.0, 1.0])

# Metodo Gradiente Conjugada PR

O método do gradiente conjugado visa alcançar uma convergência mais rápida que o método do gradiente, e ter um custo computacional menor que o método de Newton.

Considere a função  $f:\mathbb{R}^n\rightarrow\mathbb{R}$ dada por.

\

\begin{equation}
f(\textbf{x})=\frac{1}{2}\textbf{x}^{\top}A\textbf{x}+\textbf{b}^{\top}\textbf{x}+c
\end{equation}

\

Com $A\in M_{n\times n}(\mathbb{R})$ e positiva, $\textbf{b}\in \mathbb{R}^n$ e $c\in \mathbb{R}$.

Onde a função terá apenas um minimizador $x^*$ global e que satisfaz.

\

\begin{equation}
A\textbf{x}^{*}+\textbf{b}=\nabla f(x^*)=0
\end{equation}

\

Vamos tomar como direções os vetores $A$-conjugados para o método de descida.

\

Definição

Seja $A\in M_{n\times n}(\mathbb{R})$ definida positiva. Dizemos que os vetores $\textbf{d}_{0},...,\textbf{d}_{k}\in \mathbb{R^{n}}$ são $A$-conjugados se
\begin{equation}
(\textbf{d}_{i})^{\top}A\textbf{d}_{j}=0
\end{equation}
para todos $i,j=1,..,k$, com $i\neq j$

\

Lema

Seja $A\in M_{n\times n}(\mathbb{R})$ definida positiva. Um conjunto qualquer de vetore $A$-conjugados é $\textbf{L.I}$.

\

Portanto pelo lema poderemos ter $n$ direções a serem tomadas $\textbf{d}_{0},...,\textbf{d}_{n-1}$.

\

\begin{equation}
\textbf{x}_{k+1}=\textbf{x}_{k}+t_{k}\textbf{d}_{k}
\end{equation}

\

sendo $t_{k}$ o minimizador de $f(\textbf{x}_{k}+t\textbf{d}_{k})$. Portanto a derivada na direção $\textbf{d}_{k}$ no próximo ponto $\textbf{x}_{k+1}$ é nula.
\begin{equation}
\nabla f(\textbf{x}_{k+1})^{\top}\textbf{d}=0
\end{equation}
como 
\begin{equation}
\nabla f(\textbf{x}_{k+1})=A(\textbf{x}_{k}+t_{k}\textbf{d}_{k})+\textbf{b}=\nabla f(\text{x}_{k})+t_{k}A\textbf{d}_{k}
\end{equation}
Substituindo vamos obter $t_{k}$
 \begin{equation}
 t_{k}=-\frac{\nabla f(\textbf{x}_{k})^{\top}\textbf{d}_{k}}{(\textbf{d}_{k})^{\top}A\textbf{d}_{k}}
 \end{equation}

\

Vamos agora obter os $n$ vetores $A$-conjugados a partir de um $x_{0}$ dado. Defina $\textbf{d}_{0}=-\nabla f(\text{x}_{0})$, e para $k=0,...,n-2$ 
\begin{equation}
\textbf{d}_{k+1}=-\nabla f(\textbf{x}_{k+1})+\beta_{k}\textbf{d}_{k}
\end{equation}
Pela definição temos que
\begin{equation}
(\textbf{d})^{\top}_{k}A(-\nabla f(\textbf{x}_{k+1})+\beta_{k}\textbf{d}_{k})=0
\end{equation}
assim $\beta_{k}$ é dado por
\begin{equation}
\beta_{k}=\frac{(\textbf{d})^{\top}_{k}A\nabla f(\textbf{x}_{k+1})}{(\textbf{d})^{\top}_{k}A\textbf{d}_{k}}
\end{equation}
O próximo teorema nos garante que o método chegará ao mínimo de $f$.

\

Teorema

Considere a função anterior e seu minimizador $x^{*}$, definido $A\textbf{x}^{*}+\textbf{b}=\nabla f(\textbf{x}^{*})=0$. Dado $x_{0}\in \mathbb{R}^{n}$, a sequência finita definida por $\textbf{x}_{k+1}=\textbf{x}_{k}+t_{k}\textbf{d}_{k}$ cumpre $\textbf{x}_{n}=\textbf{x}^{*}$

\

Podemos fazer uma modificação para que o método economize no custo computacional no cálculo da hessiana, usando somente informações de primeira ordem. Essa variação do método funcionaria para qualquer função diferenciável, e tal modificação é dada por Polak e Ribiére como:
\begin{equation}
  \beta_{k-1}=\frac{<\nabla f(\textbf{x}_{k+1}), (\nabla f(\textbf{x}_{k+1})-\nabla f(\textbf{x}_{k}))>}{<\nabla f(\textbf{x}_{k}),\nabla f(\textbf{x}_{k})>}
\end{equation}

In [ ]:
function gradient_conjugate_PR(f::Function,x::Vector,linear_method::Function;ϵ=1e-9,n_iter=100)
    k = 0
    n = length(x)
    g_new=∇(f,x)
    d = -g_new
    while(norm(g_new)> ϵ&&k<n_iter)
        x_old = x
        g_old = g_new
        x=linear_search(f,x,d,linear_method)[2]
        g_new  = ∇(f,x)
        β = max((transpose(g_new)*(g_new - g_old))/(transpose(g_old)*g_old),0)
        d = -g_new + β*d
        k = k + 1
    end
    return (f(x),x)
end

gradient_conjugate_PR (generic function with 1 method)

In [ ]:
gradient_conjugate_PR(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[0,1],newton)

(0.0, [2.0, 1.0])

# Modo de Usar

   Sejam os dados de entrada $x \in \mathcal{R}^n e f: \mathcal{R}^n \to \mathcal{R}$. Onde $x$ representa as variáveis de decisão e $f$ a função objetivo. Vamos mostrar como chamar os métodos de otimização irrestrita(no caso temos as funções: *gradient_descent, newton_search, gradient_conjugate_PR e quasi_newton_BFGS*) para resolver o problema de encontrar um ponto ótimo e  valor da função neste ponto.

   Considere o seguinte exemplo, seja $f(x_1,x_2)= \frac{1}{2}(x_1 - 2)^2 + (x_2 - 1)^2$, $x = \begin{pmatrix}1 \\0\end{pmatrix}$ e $d  = \begin{pmatrix}3 \\1\end{pmatrix}$. Esses podem ser passados para um método da seguinte forma:
 




In [ ]:
gradient_descent(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[1,0],secao_aurea)

(9.129599700888681e-20, [1.9999999995972084, 1.0000000001008735])



   Onde o primeiro campo da função implementa uma função anônima para representar $f(x)$, com formato <i>(vetores)$\to$ (função expressa)</i>. Devemos utilizar de operações aritméticas padrões sobre um vetor[$i$], que refere-se a coordenada $i$ do vetor(em julia começa de 1 até n), para expressar $f(x)$. O campo seguinte dp método faz referência à $x$ implementado como um Array'([$x_1$,$x_2$]. 

  No último campo temos uma função referenciando o método de busca linear(no caso seção aurea) usado, temos três referências implementadas secao_aurea, armijo e newton. Elas podem ser chamadas da mesma forma do exemplo anterior, que utiliza parâmetros definidos pelo livro-texto da disciplina, ou podemos utilizar funções anonimas para chamar um método customizado da forma que foi definida no [Projeto 1](https://colab.research.google.com/drive/1Nt6ffNGr-3ej0gCgIW2sG5DYSnJ7Pywh?usp=sharing). Veja que agora nesse projeto além dos parâmetros adicionais para os métodos de busca linear, o mesmo ocorre para os métodos de otimização irrestrita.

In [ ]:
gradient_descent(x->0.5*(x[1]-2)^2 + (x[2] - 1)^2 ,[1,0],(f,x,d)->armijo(f,x,d,t=0.7,η = 0.5),ϵ=1e-4)

(1.8288426094768024e-9, [1.999946882144, 0.999979552768])

 Na proxima seção discutimos os parâmetros para cada método. A saída do método  retorna uma tupla da forma ($f(x^*),x^*$), ou seja, o valor da função objetivo no melhor ponto encontrado e o ponto.

# Parâmetros dos Métodos

   Aqui colocaremos as funções dos métodos implementados e a descrição sobre seus parâmetros. Parametros depois do caracter ";" são chamados de argumentos palavra-chave. Esses devem vir depois dos argumentos posicionais e da forma (palavra-chave) = valor. Algumas palavras-chave estão em unicode para acessar estas no julia basta escrever da mesma forma que em latex(ex. \epsilon) e pressionar a tecla 'tab', ou seja , $\epsilon$ = \epsilona + tab. No ambiente Colab para inserir os caracteres basta copiar das funções, pois o mesmo não possui suporte para inserir caracteres unicode.


gradient_descent(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)
>  Algoritmo de Gradiente Descendente clássico.

*  ϵ(\epsilon) -  tolerância utilizada para verificar convergencia. Com menor precisão mais rápido converge. 


---

newton_search(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)
> Método de Newton clássico tal que não utiliza aproximação da inversa da hessiana. No caso, tentamos utilizar as bibliotecas aqui expostas para calcular com precisão.

*  ϵ(\epsilon) -  tolerância utilizada para verificar convergencia. Com menor precisão mais rápido converge. 

---
quasi_newton_BFGS(f::Function,x::Vector,linear_method::Function;ϵ=1e-9)

>  Algoritmo de Quase Newton usando método de Broyden, Fletcher, Goldfarb e Shanno(BFGS) para calcular uma aproximação da inversa da hessiana. Utiliza tecnica de escalonamento da matriz hessiana inicial como um fator multiplicativo da identidade.

*  ϵ(\epsilon) -  tolerância utilizada para verificar convergencia. Com menor precisão mais rápido converge.

---
gradient_conjugate_PR(f::Function,x::Vector,linear_method::Function;ϵ=1e-9,n_iter=100)
>  Algoritmo de Gradiente Conjugado usando método de Polak e Ribiére para calcular direções conjugadas. Utiliza tecnica de reinicialização com o método da gradiente caso $\beta_k$ sejá negativo. Como a função de entrada não é necessariamente quadratica limitamos o número de iterações.


*  ϵ(\epsilon) -  tolerância utilizada para verificar convergencia. Com menor precisão mais rápido converge.
* n_iter - número máximo de iterações que o método pode executar.

---






Todos os métodos retornam a função objetivo e o melhor ponto encontrado. Vale ressaltar que podemos passar em linear_method uma função de passo constante ou funções com passos customizados aos métodos.

#Testes

Nesta seção definimos as instâncias de testes que usamos e quais os resultados os métodos encontraram.

<h2>Instâncias</h2>
   As instâncias dadas são definidas em termo de função objetivo, valor inicial de $x$ e direção de descida $d$.  Abaixo apresentamos uma lista com os 5 valores entrada usados.
    



1.   <i>instância 1</i>
   *   $f_1(x)=(x_1 -2)^4 + (x_1 - 2x_2)^2$
   *   $x_0 = \begin{pmatrix}0 \\ 3\end{pmatrix}$
   *   $d_0 = \begin{pmatrix}-\nabla f_1(x_0)\end{pmatrix}$
2.   <i>instância 2</i>
   *   $f_2(x)=100(x_2 -x_1^2)^2 + (1 - x_1)^2$
   *   $x_0 = \begin{pmatrix}-1,9\\2\end{pmatrix}$
   *   $d_0 = \begin{pmatrix}-\nabla f_2(x_0)\end{pmatrix}$
3.   <i>instância 3</i>
   *   $f_3(x)=1,5 - x_1(1-x_2)^2 + (2,25 - x_1(1-x_2^2)^2+(2,625-x_1(1-x_2^3))^2$
   *   $x_0 = \begin{pmatrix}0\\0\end{pmatrix}$
   *   $d_0 = \begin{pmatrix}-\nabla f_3(x_0)\end{pmatrix}$
4.   <i>instância 4</i>
   *   $f_4(x)=(x_1 - 2x_2+5x_2^2 - x_2^3 -13)^2 + (x_1 - 14x_2 + x_2^2 + x_2^3 -29)^2$
   *   $x_0 = \begin{pmatrix}15\\-2\end{pmatrix}$
   *   $d_0 = \begin{pmatrix}-\nabla f_4(x_0)\end{pmatrix}$
5.   <i>instância 5</i>
   *   $f_5(x)=100(x_3 - {(\frac{x_1+x_2}{2})}^2)^2 + (1-x_1)^2 + (1-x_2)^2$
   *   $x_0 = \begin{pmatrix}-1,2\\2\\0\end{pmatrix}$
   *   $d_0 = \begin{pmatrix}-\nabla f_5(x_0)\end{pmatrix}$








In [ ]:
#funções objetivo
f1 = x-> (x[1]-2)^4 + (x[1] - 2*x[2])^2
f2 = x-> 100*(x[2]-(x[1])^2)^2 + (1 - x[1])^2
f3 = x-> 1.5 - x[1]*(1-x[2])^2 + (2.25 - x[1]*(1-(x[2])^2))^2 +(2.625 - x[1]*(1-x[2]^3))^2
f4 = x-> (x[1] - 2*x[2]+5*x[2]^2 - x[2]^3 -13)^2 + (x[1] - 14*x[2] + x[2]^2 + x[2]^3 -29)^2
f5 = x-> 100*(x[3] - ((x[1]+x[2])/2)^2)^2 + (1-x[1])^2 + (1-x[2])^2

#valores de x indexado por função
x0 = [[0,3],[-1.9,2],[0,0],[15,-2],[-1.2,2,0]]


5-element Array{Array{Float64,1},1}:
 [0.0, 3.0]
 [-1.9, 2.0]
 [0.0, 0.0]
 [15.0, -2.0]
 [-1.2, 2.0, 0.0]

##Resultados

Abaixo mostramos o resultados usando os algoritmos do livro-texto com os valores indicados.


In [ ]:
functions = [f1,f2,f3,f4,f5]
linear_methods = [secao_aurea,newton,armijo]
methods = [gradient_descent,newton_search,gradient_conjugate_PR,quasi_newton_BFGS]
linear_methods_names = ["Seção Aurea","Método de Newton","Condição de Armijo"]
methods_names = ["Método da Gradiente","Método de Newton","Método do Gradiente Conjugado","Método de Quase Newton"]
println("----------------------------------------------------------------")
for i = 1:5
    println("instancia ",i,":")
    println("----------------------------------------------------------------")
    for j = 1:3
    print("Método de Busca Linear Usado: ")
       println(linear_methods_names[j])
       println("----------------------------------------------------------------")
       for k = 1:4
         println(methods_names[k])
         (best_obj,best_point) = methods[k](functions[i],x0[i],linear_methods[j],ϵ=1e-4) 
         print("Melhor solução : ")
         println(best_point)
         print("Valor de f na solução : ")
         println(best_obj)
         println("----------------------------------------------------------------")
       end
    end
end

----------------------------------------------------------------
instancia 1:
----------------------------------------------------------------
Método de Busca Linear Usado: Seção Aurea
----------------------------------------------------------------
Método da Gradiente
Melhor solução : [2.0283997404430587, 1.0142108251982414]
Valor de f na solução : 6.50995298024256e-7
----------------------------------------------------------------
Método de Newton
Melhor solução : [1.9945816244848118, 0.9973003366637665]
Valor de f na solução : 1.2247971485622868e-9
----------------------------------------------------------------
Método do Gradiente Conjugado
Melhor solução : [2.0081473404962593, 1.004066056213807]
Valor de f na solução : 4.6380865801980474e-9
----------------------------------------------------------------
Método de Quase Newton
Melhor solução : [2.0081473470237348, 1.0040660594522226]
Valor de f na solução : 4.63810224323956e-9
------------------------------------------------------

Podemos ver que conseguimos convergir para ótimos globais na maioria das instâncias. Porém veja que em instâncias onde temos fatores não quadráticos, o algoritmo de newton e gradiente conjugado podem não convergir para um minimo global. Outro fator que pode interferir são os métodos de busca linear, como pode ser visto, na instância 4 o método da seção áurea gerou um mínimo local que é pior para todos os algoritmos. Fato dado que o método da seção áurea possui dependência por intervalos unimodais.

Para melhorar a convergência podemos utilizar condições para busca linear mais fortes como a de Wolfe. E utilizar métodos modificados para lidar com carateristicas específicas de funções.

#Referências
 $[1]$ **Ademir A.R.** e **Elizabeth W.K.** (2011). *Um Curso De Otimização. Curitiba.*